In [ ]:
! pip install pdfplumber

### **Open the PDF and save it as text file**

In [ ]:
import pdfplumber

pdf_path = "/content/sample pdf.pdf"
output_text_file = "extracted_text.txt"

with pdfplumber.open(pdf_path) as pdf:
  extracted_text = ""
  for page in pdf.pages:
    extracted_text += page.extract_text()

with open(output_text_file, "w") as text_file:
  text_file.write(extracted_text)

print(f"Text extracted and saved to {output_text_file}")

Text extracted and saved to extracted_text.txt


### **Previewing the extracted text**

In [ ]:
with open("/content/extracted_text.txt", "r") as file:
  document_text = file.read()

print(document_text[:500])

USENIX Example Paper
Pekka Nikander Jane-Ellen Long
Aalto University USENIX Association
Abstract
This is an example for a USENIX paper, in the form
of an HTML/CSS template. Being heavily self-ref-
erential, this template illustrates the features in-
cluded in this template. It is expected that the
prospective authors using HTML/CSS would create
a new document based on this template, remove
the content, and start writing their paper.
Note that in this template, you may have a mul-
ti-paragraph ab


### **Summarizing the text**

In [ ]:
! pip install transformers

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model = "t5-small")
summary = summarizer(document_text[:1000], max_length=150, min_length=30, do_sample=False)

print("Summary:", summary[0]['summary_text'])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Summary: a USENIX paper is an example for a paper in the form of an HTML/CSS template . it is expected that the prospective authors using this template would create a new document based on this template, remove the content, and start writing their paper . the authors are solely responsible for the content and formatting of their paper.


**Splitting the document into paragraph and sentences**

In [ ]:
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize

# split text into sentences
sentences = sent_tokenize(document_text)

# combine sentences into passages
passages = []
current_passage = ""
for sentence in sentences:
    if len(current_passage.split()) + len(sentence.split()) < 150:
        current_passage += " " + sentence
    else:
        passages.append(current_passage.strip())
        current_passage = sentence
if current_passage:
    passages.append(current_passage.strip())

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


**Generating questions from the passage**

In [ ]:
qg_pipeline = pipeline("text2text-generation", model="valhalla/t5-base-qg-hl")

# function to generate questions using the pipeline
def generate_questions_pipeline(passage, min_questions=3):
    input_text = f"generate questions: {passage}"
    results = qg_pipeline(input_text)
    questions = results[0]['generated_text'].split('<sep>')

    # ensure we have at least 3 questions
    questions = [q.strip() for q in questions if q.strip()]

    if len(questions) < min_questions:
        passage_sentences = passage.split('. ')
        for i in range(len(passage_sentences)):
            if len(questions) >= min_questions:
                break
            additional_input = ' '.join(passage_sentences[i:i+2])
            additional_results = qg_pipeline(f"generate questions: {additional_input}")
            additional_questions = additional_results[0]['generated_text'].split('<sep>')
            questions.extend([q.strip() for q in additional_questions if q.strip()])

    return questions[:min_questions]

# generate questions from passages
for idx, passage in enumerate(passages):
    questions = generate_questions_pipeline(passage)
    print(f"Passage {idx+1}:\n{passage}\n")
    print("Generated Questions:")
    for q in questions:
        print(f"- {q}")
    print(f"\n{'-'*50}\n")

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/15.0 [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Device set to use cpu


Passage 1:
USENIX Example Paper
Pekka Nikander Jane-Ellen Long
Aalto University USENIX Association
Abstract
This is an example for a USENIX paper, in the form
of an HTML/CSS template. Being heavily self-ref-
erential, this template illustrates the features in-
cluded in this template. It is expected that the
prospective authors using HTML/CSS would create
a new document based on this template, remove
the content, and start writing their paper. Note that in this template, you may have a mul-
ti-paragraph abstract. However, that it is not nec-
essarily a good practice. Try to keep your abstract
in one paragraph, and remember that the optimal
length for an abstract is 200-300 words. 1 Introduction
For the purposes of USENIX conference publica-
tions, the authors, not the USENIX staff, are solely
responsible for the content and formatting of their
paper.

Generated Questions:
- What is the best length for an abstract for a USENIX paper?
- What is an example for a USENIX paper?
- What featu

 **Answer the Generated Questions Using a QA Model**

In [ ]:
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

# function to track and answer only unique questions
def answer_unique_questions(passages, qa_pipeline):
    answered_questions = set()

    for idx, passage in enumerate(passages):
        questions = generate_questions_pipeline(passage)

        for question in questions:

           # check if the question has already been answered
            if question not in answered_questions:
                answer = qa_pipeline({'question': question, 'context': passage})
                print(f"Q: {question}")
                print(f"A: {answer['answer']}\n")
                answered_questions.add(question)
        print(f"{'='*50}\n")

answer_unique_questions(passages, qa_pipeline)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cpu
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/question_answering.py:395: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


Q: What is the best length for an abstract for a USENIX paper?
A: 200-300 words

Q: What is an example for a USENIX paper?
A: USENIX Association
Abstract

Q: What features are included in the HTML/CSS template?
A: 
cluded


Q: What is the purpose of this template?
A: to help


Q: What is the purpose of the Hkon Wium Lie?
A: a guide to using plate

Q: What did Jane-Ellen Long help write a guide to using plate FrameMaker for USENIX papers?
A: Pekka
Nikander


Q: What is the purpose of the templates provided by USENIX organization is to lift the burden of caring by Justin Zobel?
A: Writing for Computer Science

Q: What is the name of the author of The Elements of Style by Strunk?
A: Justin Zobel

Q: What does White require understanding of style and typography?
A: White[1]is a classic, and has remained a bestseller



Q: What is the name of the templateWeb E-Mail Tuner Menu Previous Next Mute -Volume +Volume Esc F1 F2 F3 F4 F5 F6 F7 F8 F9 F10 F11 F12 Power Sleep W Up ake Num Lock Caps Loc

/usr/local/lib/python3.11/dist-packages/transformers/pipelines/question_answering.py:395: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


Q: What is the name of the item that floats to the top of the page?
A: Figure 2

Q: What is the name of the figure that floats to the top of the page?
A: Figure 2

Q: What is the name of the character that floats to the top of the page?
A: Z X C V B N M


Q: What does the template use to process refer- provide styling?
A: JavaScript

Q: Who adds to the Proceedings?
A: those who compile the Proceedings


Q: What do browsers support in order to convert a document to PDF?
A: a for-
figure element

Q: What type of caption element is needed to convert a document to PDF?
A: for-
figure element

Q: What does Theciteelement hold all ref- functionality for page-based formatting?
A: CSS
captionelement


Q: What is CSS used to achieve the presentation of USENIX $ prince --javascript example.html papers?
A: multi-column layout 4 Tables
• footnotes


Q: What is used to present USENIX $ prince --javascript example.html papers?
A: CSS


Q: What is the name of the project that created USENET?
A: The S

### **Streamlit app**

In [ ]:
! pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.5 MB/s eta 0:00:00


In [ ]:
! pip install pyngrok

In [ ]:
%%writefile app.py

import streamlit as st
import pdfplumber
from transformers import pipeline
import nltk
from nltk.tokenize import sent_tokenize
import time

try:
    nltk.data.find('tokenizers/punkt')
except nltk.downloader.DownloadError:
    nltk.download('punkt')

st.set_page_config(
    page_title="PDF Analysis Tool",
    layout="wide",
    initial_sidebar_state="expanded"
)

st.markdown("""
    <style>
        .main {
            background-color: #f8f9fa;
        }
        .stApp {
            background-color: #ffffff;
        }
        .header {
            font-size: 2.2rem;
            font-weight: 600;
            color: #1a365d;
            text-align: center;
            margin-bottom: 0.5rem;
        }
        .subheader {
            font-size: 1rem;
            color: #4a5568;
            text-align: center;
            margin-bottom: 2rem;
        }
        .card {
            background-color: white;
            border-radius: 8px;
            padding: 1.5rem;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            margin-bottom: 1.5rem;
            border-left: 4px solid #3182ce;
        }
        .card-title {
            font-size: 1.2rem;
            font-weight: 600;
            color: #2c5282;
            margin-bottom: 1rem;
        }
        .success-message {
            background-color: #ebf8ff;
            border: 1px solid #bee3f8;
            border-radius: 6px;
            padding: 1rem;
            margin: 1rem 0;
        }
        .question-item {
            background-color: #f7fafc;
            border-left: 3px solid #90cdf4;
            padding: 0.8rem;
            margin: 0.5rem 0;
            border-radius: 0 6px 6px 0;
        }
        .answer-box {
            background-color: #ebf8ff;
            border: 1px solid #bee3f8;
            border-radius: 6px;
            padding: 1.2rem;
            margin-top: 1rem;
        }
        .file-uploader {
            border: 2px dashed #cbd5e1;
            border-radius: 8px;
            padding: 2rem 1rem;
            text-align: center;
        }
        .highlight {
            background-color: #ebf8ff;
            padding: 0.2rem 0.4rem;
            border-radius: 4px;
            font-weight: 500;
        }
    </style>
""", unsafe_allow_html=True)

# App Header
st.markdown('<div class="header">PDF Analysis Tool</div>', unsafe_allow_html=True)
st.markdown("""
    <div class="subheader">
        Analyze PDF documents with automated text processing, summarization, and question answering
        powered by natural language processing models.
    </div>
""", unsafe_allow_html=True)

# Features section
with st.expander("Features", expanded=True):
    col1, col2, col3 = st.columns(3)
    with col1:
        with st.container():
            st.markdown('<div class="card-title">Text Extraction</div>', unsafe_allow_html=True)
            st.markdown('Extract clean text from PDF documents while preserving structure')
    with col2:
        with st.container():
            st.markdown('<div class="card-title">Content Summarization</div>', unsafe_allow_html=True)
            st.markdown('Generate concise summaries using transformer-based models')
    with col3:
        with st.container():
            st.markdown('<div class="card-title">Question Answering</div>', unsafe_allow_html=True)
            st.markdown('Automatically generate and answer questions about the content')

# File upload section
with st.container():
    st.markdown('<div class="card-title">Upload Document</div>', unsafe_allow_html=True)
    uploaded_file = st.file_uploader(
        "Select a PDF file for analysis",
        type=["pdf"],
        label_visibility="collapsed"
    )

# Initialize session state
if 'extracted_text' not in st.session_state:
    st.session_state.extracted_text = ""
if 'generated_questions' not in st.session_state:
    st.session_state.generated_questions = []
if 'summary' not in st.session_state:
    st.session_state.summary = ""

# Process uploaded file
if uploaded_file is not None:
    with st.spinner("Extracting text from PDF document..."):
        progress_bar = st.progress(0)
        st.session_state.extracted_text = ""

        with pdfplumber.open(uploaded_file) as pdf:
            total_pages = len(pdf.pages)
            for i, page in enumerate(pdf.pages):
                st.session_state.extracted_text += page.extract_text() or ""
                progress_bar.progress((i + 1) / total_pages)
                time.sleep(0.05)

        progress_bar.empty()

        st.markdown('<div class="success-message">Text extracted successfully</div>', unsafe_allow_html=True)

        with st.expander("View Extracted Text", expanded=False):
            st.text_area(
                "Extracted Text Preview:",
                st.session_state.extracted_text[:1500] + ("..." if len(st.session_state.extracted_text) > 1500 else ""),
                height=300,
                label_visibility="collapsed"
            )

    # Analysis features
    tab1, tab2, tab3 = st.tabs(["Summary", "Questions", "Question Answering"])

    with tab1:
        with st.container():
            st.markdown('<div class="card-title">Document Summary</div>', unsafe_allow_html=True)
            st.markdown('Generate a condensed version of the document highlighting key information')

            if st.button("Generate Summary", key="summary_btn"):
                if st.session_state.extracted_text:
                    with st.spinner("Processing document content..."):
                        def summarize_text(text):
                            summarizer = pipeline("summarization", model="t5-small")
                            chunk_size = 1000
                            summaries = []
                            for i in range(0, len(text), chunk_size):
                                chunk = text[i:i+chunk_size]
                                summary = summarizer(chunk, max_length=150, min_length=30, do_sample=False)
                                summaries.append(summary[0]['summary_text'])
                            return " ".join(summaries)

                        st.session_state.summary = summarize_text(st.session_state.extracted_text)

                    st.markdown('<div class="card-title">Generated Summary</div>', unsafe_allow_html=True)
                    st.markdown(f'<div class="answer-box">{st.session_state.summary}</div>', unsafe_allow_html=True)
                else:
                    st.warning("Please upload a valid PDF document first.")

    with tab2:
        with st.container():
            st.markdown('<div class="card-title">Question Generation</div>', unsafe_allow_html=True)
            st.markdown('Automatically generate relevant questions about the document content')

            if st.button("Generate Questions", key="questions_btn"):
                if st.session_state.extracted_text:
                    with st.spinner("Analyzing document to generate questions..."):
                        def generate_questions(text):
                            qg_pipeline = pipeline("text2text-generation", model="valhalla/t5-base-qg-hl")
                            sentences = sent_tokenize(text)
                            passages = []
                            current_passage = ""
                            for sentence in sentences:
                                if len(current_passage.split()) + len(sentence.split()) < 150:
                                    current_passage += " " + sentence
                                else:
                                    passages.append(current_passage.strip())
                                    current_passage = sentence
                            if current_passage:
                                passages.append(current_passage.strip())

                            all_questions = []
                            for passage in passages:
                                input_text = f"generate questions: {passage}"
                                results = qg_pipeline(input_text)
                                questions = results[0]['generated_text'].split('<sep>')
                                all_questions.extend([q.strip() for q in questions if q.strip()])

                            return all_questions

                        st.session_state.generated_questions = generate_questions(st.session_state.extracted_text)

                    st.markdown('<div class="card-title">Generated Questions</div>', unsafe_allow_html=True)
                    if st.session_state.generated_questions:
                        for question in st.session_state.generated_questions:
                            st.markdown(f'<div class="question-item">{question}</div>', unsafe_allow_html=True)
                    else:
                        st.info("No questions could be generated from this document.")
                else:
                    st.warning("Please upload a valid PDF document first.")

    with tab3:
        with st.container():
            st.markdown('<div class="card-title">Question Answering</div>', unsafe_allow_html=True)
            st.markdown('Get answers to questions about the document content')

            if st.session_state.generated_questions:
                selected_question = st.selectbox(
                    "Select a generated question:",
                    st.session_state.generated_questions,
                    index=0
                )

                if st.button("Get Answer", key="answer_btn"):
                    if st.session_state.extracted_text and selected_question:
                        with st.spinner("Searching for answer in document..."):
                            def answer_question(question, context):
                                qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")
                                answer = qa_pipeline({'question': question, 'context': context})
                                return answer['answer']

                            answer = answer_question(selected_question, st.session_state.extracted_text)

                            st.markdown('<div class="card-title">Answer</div>', unsafe_allow_html=True)
                            st.markdown(f'<div class="answer-box">{answer}</div>', unsafe_allow_html=True)
                    else:
                        st.warning("Please generate questions first.")
            else:
                st.info("Generate questions first to use the question answering feature.")

else:
    st.info("Please upload a PDF document to begin analysis.")

# Footer
st.markdown("---")
st.markdown("""
    <div style="text-align: center; color: #718096; font-size: 0.9rem;">
        PDF Analysis Tool | Powered by Transformers and Streamlit
    </div>
""", unsafe_allow_html=True)

Overwriting app.py


In [ ]:
from pyngrok import ngrok
import os

ngrok.set_auth_token("")

# Kill any previous tunnels
ngrok.kill()

# Running streamlit app in background
os.system("streamlit run app.py &")

# Creating a public tunnel
public_url = ngrok.connect(8501, "http")
print("Public URL:", public_url)

Public URL: NgrokTunnel: "https://65fd55acc58c.ngrok-free.app" -> "http://localhost:8501"
